In [1]:
import pandas as pandas
import numpy as np
import matplotlib.pyplot as plt
import flwr as fl
#tensorflow se usa para construir y entrenar modelos de aprendizaje automático, redes neuronales.
#keras  igual que tensorflow es una biblioteca de aprendizaje automático.
import tensorflow as tf
from tensorflow import keras
from keras import layers
import threading

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np

# ----------------------
# Dataset
# ----------------------
# Cargar y preprocesar el dataset MNIST, usando solo una parte para acelerar
#MNIST es una base de datos de imágenes de dígitos escritos a mano, comúnmente utilizada para entrenar y probar modelos de aprendizaje automático.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
#Normaliza los píxeles de 0 a 255 a un rango de 0 a 1 dividiendo por 255.0
x_train, x_test = x_train[:1000] / 255.0, x_test[:200] / 255.0
#USsa solo los primeros 1000 ejemplos de entrenamiento y 200 ejemplos de prueba para acelerar el proceso.
y_train, y_test = y_train[:1000], y_test[:200]

# ----------------------
# Modelo
# ----------------------
# Construir el modelo
def build_model():
    model = keras.Sequential([
        #Flatten convierte una matriz 2D (28x28 píxeles) en un vector 1D (784 píxeles)
        layers.Flatten(input_shape=(28,28)),
        #dense es una capa completamente conectada
        #relu es una función de activación que introduce no linealidad
        layers.Dense(64, activation="relu"),
        #softmax convierte las salidas en probabilidades para clasificación multiclase
        layers.Dense(10, activation="softmax")
    ])
    #compila el modelo con el optimizador Adam, la función de pérdida de entropía cruzada categórica y la métrica de precisión
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

# ----------------------
# Simulación de entrenamiento federado local
# ----------------------
clients = 3
rounds = 10
#En este punto, se inicializan los pesos globales del modelo, que significan los parámetros del modelo antes de cualquier entrenamiento.
#Estos pesos se distribuirán a los clientes para que puedan entrenar localmente.
global_weights = build_model().get_weights()

# Según la ronda que hemos indicado anteriormente
for rnd in range(rounds):
    # Entrenamiento local en cada cliente
    #Aquí, cada cliente recibe los pesos globales, entrena su modelo localmente con sus datos y luego devuelve los pesos actualizados.
    local_weights = []
    for c in range(clients):
        model = build_model()
        model.set_weights(global_weights)
        #se entrena el modelo localmente durante 1 época con un tamaño de lote de 32
        model.fit(x_train, y_train, epochs=1, batch_size=32, verbose=0)
        #Guarda los pesos actualizados después del entrenamiento local
        local_weights.append(model.get_weights()) 
    
    # Media simple de pesos
    new_weights = []
    #Aquí, los pesos de todos los clientes se promedian para obtener los nuevos pesos globales.
    for weights in zip(*local_weights):
        new_weights.append(np.mean(weights, axis=0))
    
    global_weights = new_weights
    
    # Evaluar
    model.set_weights(global_weights)
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"Ronda {rnd+1} - Accuracy global: {acc:.4f}")






Ronda 1 - Accuracy global: 0.6950
Ronda 2 - Accuracy global: 0.7800
Ronda 3 - Accuracy global: 0.8450
Ronda 4 - Accuracy global: 0.8800
Ronda 5 - Accuracy global: 0.9200
Ronda 6 - Accuracy global: 0.9100
Ronda 7 - Accuracy global: 0.9250
Ronda 8 - Accuracy global: 0.9300
Ronda 9 - Accuracy global: 0.9400
Ronda 10 - Accuracy global: 0.9300
